**Практический пример #3** \\
Доклад: Структурированная нормализация текста с использованием недетерминированных FST \\
Автор: Владимир Марков ([@markovvn1](https://t.me/markovvn1))

In [ ]:
!pip install pynini==2.1.6

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 MB 2.6 MB/s eta 0:00:00


In [ ]:
import pynini

In [ ]:
def draw(fst):
  ascii_table = pynini.SymbolTable()
  ascii_table.add_symbol("<eps>", 0)
  for i in range(12, 128):
    ascii_table.add_symbol(chr(i), i)
  fst.set_input_symbols(ascii_table)
  fst.set_output_symbols(ascii_table)
  return fst

In [ ]:
hours = pynini.cross("01:", "один час") | pynini.cross("02:", "два часа") | pynini.cross("03:", "три часа")
minutes = pynini.cross("00", "") | pynini.cross("30", " тридцать минут")
normalize_time_fst = (hours + minutes).optimize()

convert_AM_hours = pynini.union(*[pynini.cross(f"{i:02d}", f"{i % 12:02d}") for i in range(1, 13)])
convert_PM_hours = pynini.union(*[pynini.cross(f"{i:02d}", f"{i % 12 + 12:02d}") for i in range(1, 13)])
any_minutes = pynini.union(*"012345") + pynini.union(*"0123456789")
convert_time_fst = (convert_AM_hours + ":" + any_minutes + pynini.cross(" AM", "")) | (convert_PM_hours + ":" + any_minutes + pynini.cross(" PM", ""))
convert_time_fst = convert_time_fst.optimize()

In [ ]:
sigma_star = pynini.union(*[f"[{i}]" for i in range(1, 256)]).closure()
normalize_time_cdr = pynini.cdrewrite(normalize_time_fst, "", "", sigma_star)
convert_time_cdr = pynini.cdrewrite(convert_time_fst, pynini.union(*" ", "[BOS]"), pynini.union(*" ,.", "[EOS]"), sigma_star)
fst = pynini.compose(convert_time_cdr, normalize_time_cdr).optimize()

In [ ]:
print(("Завтра в 03:00 в парке" @ fst).string())
print(("В 01:30 AM в парке" @ fst).string())
print(("Агент 02 ждет в 02:00" @ fst).string())
print(("Агент 02 ждет в 12:30 AM" @ fst).string())

Завтра в три часа в парке
В один час тридцать минут в парке
Агент 02 ждет в два часа
Агент 02 ждет в 00:30
